In [ ]:
# ╭──────────────────────────────────────────────────────────────────────────────────────────────────────────╮
# │                          Library/Module Imports and Global Variable Initialization                       │
# ╰──────────────────────────────────────────────────────────────────────────────────────────────────────────╯
import ipywidgets as widgets
from IPython.display import display, clear_output
import json
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from matplotlib import patches
from matplotlib.ticker import AutoMinorLocator
import numpy as np
import pandas as pd
from pathlib import Path

df = pd.DataFrame()
pressure_df = pd.DataFrame()
step_changes = pd.DataFrame()
step_df = pd.DataFrame()
step_times, step_labels, unique_steps = [], [], []
steady_frac = 0.1
slope_tol_s = 15.0
amp_tol = 100
min_points = 5
step_dict = {
    "Wait Before Start": {'color':'#dab1da', 'outline':'#986998', 'marker':'s'},
    "Predose Pump":      {'color':'#f0d3ef', 'outline':'#a881a7', 'marker':'o'},
    "N2Overlap":         {'color':'#c9d9ec', 'outline':'#8b96a4', 'marker':'s'},
    "PumpOverlap":       {'color':'#dceaf7', 'outline':'#9aa3ac', 'marker':'o'},
    "Open Time":         {'color':'#b9e0c6', 'outline':'#7f9c88', 'marker':'s'},
    "Hold":              {'color':'#d5edd8', 'outline':'#95a59a', 'marker':'o'},
    "Pump Purge":        {'color':'#fbd2a5', 'outline':'#b28774', 'marker':'s'},
    "Purge":             {'color':'#ffe0cb', 'outline':'#b29c8e', 'marker':'o'}
}

# ╭──────────────────────────────────────────────────────────────────────────────────────────────────────────╮
# │                           Combined File Loader (Upload or Path) + Max Pressure                           │
# ╰──────────────────────────────────────────────────────────────────────────────────────────────────────────╯
upload_mode_button = widgets.Button(description="Upload File", layout=widgets.Layout(width='150px'))
path_mode_button = widgets.Button(description="File Path", layout=widgets.Layout(width='150px'))
file_upload = widgets.FileUpload(accept='.json', multiple=False, description='Browse')

file_path_text = widgets.Text(value='', placeholder='Paste JSON file path here', description='File Path:',
                              style={'description_width': 'initial'}, layout=widgets.Layout(width='600px'))

load_button = widgets.Button(description="Load File", icon="upload", layout=widgets.Layout(width='120px'))
load_status = widgets.Label(value='')

input_box = widgets.VBox([])

def show_upload_mode(button=None):
    upload_mode_button.button_style = 'info'
    path_mode_button.button_style = ''
    input_box.children = [file_upload]
    
def show_path_mode(button=None):
    upload_mode_button.button_style = ''
    path_mode_button.button_style = 'info'
    input_box.children = [widgets.HBox([file_path_text, load_button])]

upload_mode_button.on_click(show_upload_mode)
path_mode_button.on_click(show_path_mode)

def process_data(data, filename):
    global df, pressure_df, step_changes, step_times, step_labels, unique_steps, step_df
    for entry in data:
        if isinstance(entry.get("payload"), str):
            entry["payload"] = json.loads(entry["payload"])
    df = pd.json_normalize(data)
    df["payload.TimeElapsed"] = pd.to_numeric(df["payload.TimeElapsed"], errors="coerce")
    df["payload.Pressure"] = pd.to_numeric(df["payload.Pressure"], errors="coerce")
    df["payload.CurrentCycle"] = pd.to_numeric(df["payload.CurrentCycle"], errors="coerce")
    pressure_df = df[df["type"] == "pressure"].copy()
    
    step_changes = df[df["payload.CurrentStep"].notna()].copy()
    step_changes["PreviousStep"] = step_changes["payload.CurrentStep"].shift()
    step_changes = step_changes[step_changes["payload.CurrentStep"] != step_changes["PreviousStep"]]
    step_times = step_changes["payload.TimeElapsed"].tolist()
    step_times.append(df["payload.TimeElapsed"].max())
    step_labels = step_changes["payload.CurrentStep"].tolist()
    unique_steps = list(dict.fromkeys(step_labels)) # could just hardcode this since color_map is hardcoded

    step_cycles = step_changes["payload.CurrentCycle"].astype(int).tolist()
    step_df = pd.DataFrame({"Cycle": step_cycles, "Step": step_labels,
                            "Start Time (ms)": step_times[:-1], "End Time (ms)": step_times[1:]})
    step_df["Duration (ms)"] = step_df["End Time (ms)"] - step_df["Start Time (ms)"]
    step_df["Duration (s)"] = step_df["Duration (ms)"] / 1000

    # max pressure can be calculated once when processing the file because it doesn't depend on any adjustable parameters,
    # unlike steady-state pressure, which depends on user-defined criteria like steady fraction, slope tolerance, etc.
    max_pressures = []
    for i in range(len(step_df)):
        start, end = step_df.loc[i, "Start Time (ms)"], step_df.loc[i, "End Time (ms)"]
        time_range_bool = (df["payload.TimeElapsed"] >= start) & (df["payload.TimeElapsed"] < end)
        max_pressures.append(df.loc[time_range_bool, "payload.Pressure"].max())
    step_df["Max Pressure (mTorr)"] = max_pressures
    # but we still want to calculate an initial steady-state profile based on default parameter values.
    compute_steady_pressures_and_slopes()
    
    load_status.value = f"Loaded file: {filename}"
    update_all_tabs()

def load_file_from_path(button=None):
    file_path = Path(file_path_text.value.strip().replace("\\","/"))
    if not file_path.exists():
        load_status.value = f"File not found: {file_path}"
        return
    try:
        with file_path.open('r', encoding='utf-8') as file:
            data = [json.loads(line) for line in file]
        process_data(data, str(file_path))
    except Exception as err:
        load_status.value = f"Error: {err}"
    
def load_file_from_upload(change=None):
    if not file_upload.value: # defensive programming. should never actually happen
        load_status.value = "Please upload a file."
        return
    try:
        if isinstance(file_upload.value, dict):
            uploaded = list(file_upload.value.values())[0]
            filename = uploaded['name']
            content = uploaded['content']
        else:
            uploaded = file_upload.value[0]
            filename = uploaded.name
            content = uploaded.content
        
        file_bytes = content if isinstance(content, bytes) else content.tobytes()
        data = [json.loads(line) for line in file_bytes.decode('utf-8').splitlines()]
        process_data(data, filename)
    except Exception as err:
        load_status.value = f"Error: {err}"

load_button.on_click(load_file_from_path)
file_upload.observe(load_file_from_upload, names='value')

# removed warning text about file sizes
warning_text = """
"""

file_load_warning = widgets.HTML(warning_text, layout=widgets.Layout(margin='0px 0px 0px 0px')) # optional margins
display(file_load_warning)

display(widgets.HBox([upload_mode_button, path_mode_button]))
display(input_box)
display(load_status)

show_upload_mode() # show upload mode by default on start

max_step_selector = widgets.SelectMultiple(
    options=['All'],          
    value=['All'],            
    description='Choose Step(s):',
    disabled=False,
    style={'description_width':'initial'},
    rows=9,
    layout=widgets.Layout(width='300px')
)
# ╭──────────────────────────────────────────────────────────────────────────────────────────────────────────╮
# │                                                Pressure                                                  │
# ╰──────────────────────────────────────────────────────────────────────────────────────────────────────────╯
def plot_pressure(cycle:int = None):
    fig, ax = plt.subplots(figsize=(15,5))
    ax.plot(pressure_df["payload.TimeElapsed"], pressure_df["payload.Pressure"], color='black', linewidth=2, label="Pressure")
    for i in range(len(step_labels)):
        start, stop = step_times[i], step_times[i+1]
        label = step_labels[i]
        color = step_dict[label]['color']
        ax.add_patch(patches.Rectangle((start, ax.get_ylim()[0]), stop-start,
                                       ax.get_ylim()[1]-ax.get_ylim()[0], color=color, alpha=1))
    legend_handles = [patches.Patch(color=step_dict[step]['color'], label=step) for step in unique_steps]
    ax.legend(handles=legend_handles, title="Step", bbox_to_anchor=(1.01,1), loc='upper left')
    ax.set_xlabel("Time Elapsed (ms)")
    ax.set_ylabel("Pressure (mTorr)")

    if cycle is None:
        ax.set_title("Pressure vs Time Elapsed")
        cycle_changes = df[df["payload.CurrentCycle"].notna()].copy()
        cycle_changes["PreviousCycle"] = cycle_changes["payload.CurrentCycle"].shift()
        cycle_changes = cycle_changes[cycle_changes["payload.CurrentCycle"] != cycle_changes["PreviousCycle"]]
        cycle_changes["payload.TimeElapsed"] = pd.to_numeric(cycle_changes["payload.TimeElapsed"], errors="coerce")
        for time in cycle_changes["payload.TimeElapsed"]:
            ax.axvline(x=time, color='red', linestyle='--', linewidth=2, alpha=1)
        plt.xlim([0, df["payload.TimeElapsed"].max()])
    else:
        cycle_start = df[df["payload.CurrentCycle"]==cycle]["payload.TimeElapsed"].min()
        cycle_end = df[df["payload.CurrentCycle"]==cycle+1]["payload.TimeElapsed"].min()
        if pd.isna(cycle_end):
            cycle_end = df["payload.TimeElapsed"].max()
        ax.set_title(f"Pressure vs Time Elapsed (Cycle {cycle})")
        plt.xlim([cycle_start, cycle_end])

    plt.tight_layout()
    plt.rcParams['figure.dpi'] = 200 
    plt.show()

cycle_dropdown = widgets.Dropdown(description='Choose Cycle:', style={'description_width':'initial'},
                                  layout=widgets.Layout(width='250px'))
current_cycle = None  # will initialize to first non-zero cycle after loading file

prev_cycle_button = widgets.Button(description='Previous Cycle ', layout=widgets.Layout(width='120px'))
next_cycle_button = widgets.Button(description='Next Cycle', layout=widgets.Layout(width='120px'))

def next_cycle(b):
    global current_cycle
    cycles = sorted(df['payload.CurrentCycle'].dropna().unique().astype(int).tolist()) # could just make 'cycles' a global variable
    cycles = [c for c in cycles if c > 0]  # remove cycle 0. interestingly, cycle 0 is a mirror reflection of 'All' about y-axis
    cycles = ['All'] + cycles
    start_cycle = cycle_dropdown.value if cycle_dropdown.value in cycles else cycles[0]
    idx = cycles.index(start_cycle)
    current_cycle = cycles[(idx + 1) % len(cycles)]  # wraps around. e.g., if max cycle and press 'Next Cycle' -> go back to 'All' cycle
    cycle_dropdown.value = current_cycle

def prev_cycle(b):
    global current_cycle
    cycles = sorted(df['payload.CurrentCycle'].dropna().unique().astype(int).tolist())
    cycles = [c for c in cycles if c > 0]
    cycles = ['All'] + cycles
    start_cycle = cycle_dropdown.value if cycle_dropdown.value in cycles else cycles[0]
    idx = cycles.index(start_cycle)
    current_cycle = cycles[(idx - 1) % len(cycles)]
    cycle_dropdown.value = current_cycle

prev_cycle_button.on_click(prev_cycle)
next_cycle_button.on_click(next_cycle)

cycle_selector_box = widgets.HBox([prev_cycle_button, next_cycle_button])

pressure_overlay_output = widgets.Output()

def update_pressure_overlay():
    with pressure_overlay_output:
        clear_output(wait=True)
        if pressure_df.empty:
            print("No pressure data loaded.")
            return
        fig, ax = plt.subplots(figsize=(15,5))
        cycles = sorted(pressure_df['payload.CurrentCycle'].dropna().unique().astype(int).tolist())
        if len(cycles) <= 1:
            print("Not enough cycles to plot.")
            return
        
        cmap = cm.jet
        norm = mcolors.Normalize(vmin=1, vmax=max(cycles))
        
        max_relative_time = 0 # this is so no data is excluded if there is a 'Wait Before Start' step
        
        for c in cycles[1:]:  # ignore cycle 0
            subset = pressure_df[pressure_df["payload.CurrentCycle"] == c].copy()
            t0 = subset["payload.TimeElapsed"].min()
            subset["Normalized Time"] = subset["payload.TimeElapsed"] - t0
            
            max_relative_time = max(max_relative_time, subset["Normalized Time"].max())
            
            color = cmap(norm(c))
            ax.plot(subset["Normalized Time"], subset["payload.Pressure"], linewidth=2, color=color)
        
        # Set x-axis from 0 to max relative time
        ax.set_xlim(0, max_relative_time)
        
        ax.set_xlabel("Time Elapsed Relative to Cycle Start (ms)")
        ax.set_ylabel("Pressure (mTorr)")
        ax.set_title("Overlay of All Cycles")
        
        # major and minor grids
        ax.grid(which='major', linestyle='-', linewidth=0.8, color='gray')
        ax.grid(which='minor', linestyle='--', linewidth=0.5, color='lightgray')
        
        # enable minor ticks
        ax.minorticks_on()
        
        # add a continuous colorbar to indicate cycle numbers. if it was discrete, legend would explode in size with more cycles
        sm = cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        cbar = plt.colorbar(sm, ax=ax)
        cbar.set_label('Cycle Number')
        
        plt.tight_layout()
        plt.show()

overlay_accordion = widgets.Accordion(children=[pressure_overlay_output])
overlay_accordion.set_title(0, "Overlay of All Cycles")


def update_cycle_dropdown():
    if df.empty:
        cycle_dropdown.options = ['All']
        cycle_dropdown.value = 'All'
        return

    # temporarily remove observer to prevent update_pressure_plot() from running twice when initializing dropdown options
    try:
        cycle_dropdown.unobserve(update_pressure_plot, names='value')
    except:
        pass

    # update options
    cycle_dropdown.options = ['All'] + sorted(df['payload.CurrentCycle'].dropna().unique().astype(int).tolist())[1:]
    cycle_dropdown.value = 'All'

    # re-add observer
    cycle_dropdown.observe(update_pressure_plot, names='value')

cycle_dropdown = widgets.Dropdown(description='Choose Cycle:', style={'description_width':'initial'},
                                  layout=widgets.Layout(width='250px'))

# ╭──────────────────────────────────────────────────────────────────────────────────────────────────────────╮
# │                                              Steady-State                                                │
# ╰──────────────────────────────────────────────────────────────────────────────────────────────────────────╯
def steady_pressure_for_window(times_ms, pressures):
    valid_points_array = np.isfinite(times_ms) & np.isfinite(pressures) # remove NaN values
    
    if not np.any(valid_points_array): 
        return np.nan, np.nan

    # use only valid points
    valid_times_ms = np.asarray(times_ms[valid_points_array], dtype=float)
    valid_pressures = np.asarray(pressures[valid_points_array], dtype=float)

    # need at least 2 points to compute a slope
    if valid_times_ms.size < 2: 
        return np.nan, np.nan

    # sort by increasing time for slope calculations
    order = np.argsort(valid_times_ms)
    valid_times_ms, valid_pressures = valid_times_ms[order], valid_pressures[order]

    # if multiple pressure values have the same timestamp (don't think this would ever happen anyways), average them
    unique_time, inv = np.unique(valid_times_ms, return_inverse=True)
    sums = np.bincount(inv, weights=valid_pressures)
    counts = np.bincount(inv)
    valid_pressures = sums / counts
    valid_times_ms = unique_time
    
    times_s = valid_times_ms / 1000.0
    num_points = valid_times_ms.size
    
    start_index = max(int(np.floor(num_points*(1-steady_frac))),0)
    steady_times, steady_pressures = times_s[start_index:], valid_pressures[start_index:]
    
    if steady_times.size < min_points:
        return np.nan, np.nan
    
    duration_s = steady_times[-1]-steady_times[0]
    
    if duration_s <= 0:
        return np.nan, np.nan

    if steady_times.size >= 3:
        slope_s = np.polyfit(steady_times, steady_pressures, 1)[0]
    else:
        slope_s = (steady_pressures[-1]-steady_pressures[0])/duration_s
    
    amp = float(np.nanmax(steady_pressures)-np.nanmin(steady_pressures))
    is_steady = (abs(slope_s)<=slope_tol_s) and (amp<=amp_tol)
    return (float(np.nanmean(steady_pressures)) if is_steady else np.nan, slope_s)
    
def compute_steady_pressures_and_slopes():
    global step_df
    if step_df.empty or df.empty:
        return

    steady_pressures, slopes = [], []

    for i in range(len(step_df)):
        start, end = step_df.loc[i, "Start Time (ms)"], step_df.loc[i, "End Time (ms)"]
        time_range_bool = (df["payload.TimeElapsed"] >= start) & (df["payload.TimeElapsed"] < end)
        times = df.loc[time_range_bool, "payload.TimeElapsed"].to_numpy()
        pressures = df.loc[time_range_bool, "payload.Pressure"].to_numpy()
        steady_p, slope_s = steady_pressure_for_window(times, pressures)
        steady_pressures.append(steady_p)
        slopes.append(slope_s)

    step_df["Steady Pressure (mTorr)"] = steady_pressures
    step_df["Slope (mTorr/s)"] = slopes

steady_state_patch_output = widgets.Output()

def plot_steady_state_patch(change=None):
    if pressure_df.empty or step_df.empty:
        with steady_state_patch_output:
            clear_output(wait=True)
        return

    # obtain selected cycle from dropdown
    cycle = steady_cycle_dropdown.value
    selected_steady_steps = steady_step_selector.value

    if cycle == 'All':
        pressure_subset = pressure_df
        step_subset = step_df
    else:
        cycle_int = int(cycle)
        pressure_subset = pressure_df[pressure_df["payload.CurrentCycle"] == cycle_int]
        step_subset = step_df[step_df["Cycle"] == cycle_int]
        
    if 'All' not in selected_steady_steps:
        step_subset = step_subset[step_subset["Step"].isin(selected_steady_steps)]
        
    with steady_state_patch_output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(15,5))
        ax.plot(pressure_subset["payload.TimeElapsed"], pressure_subset["payload.Pressure"], 
                color='black', linewidth=2, label="Pressure")

        # steady-state patches (note: this is a different method than pressure plot. doesn't really matter, could do either)
        ylim = ax.get_ylim() # (ymin, ymax)
        for _, row in step_subset.iterrows(): # iterrows -> (index, row), but ignore index b/c we don't need
            start, stop = row["Start Time (ms)"], row["End Time (ms)"]
            color_patch = 'green' if not np.isnan(row["Steady Pressure (mTorr)"]) else 'red'
            ax.add_patch(patches.Rectangle((start, ylim[0]), stop-start, ylim[1]-ylim[0],
                                           color=color_patch, alpha=0.3, zorder=0))

        # x-axis limits
        if cycle == 'All':
            plt.xlim([0, df["payload.TimeElapsed"].max()])
        else:
            cycle_start = pressure_subset["payload.TimeElapsed"].min()
            next_cycle_start = df[df["payload.CurrentCycle"] == (int(cycle)+1)]["payload.TimeElapsed"].min()
            if pd.isna(next_cycle_start):
                next_cycle_start = df["payload.TimeElapsed"].max()
            plt.xlim([cycle_start, next_cycle_start])

        # legend for steady / not steady
        steady_patch = patches.Patch(color='green', alpha=0.3, label='Steady')
        non_steady_patch = patches.Patch(color='red', alpha=0.3, label='Not Steady')
        ax.legend(handles=[steady_patch, non_steady_patch], bbox_to_anchor=(1.01,1), loc='upper left')

        ax.set_xlabel("Time Elapsed [ms]")
        ax.set_ylabel("Pressure [mTorr]")
        ax.set_title(f"Steady-State Classification for Cycle {cycle}) by Step")
        plt.tight_layout()
        plt.show()

def update_steady_cycle_dropdown():
    if df.empty: # e.g., before loading a file
        steady_cycle_dropdown.options = ['All']
        steady_cycle_dropdown.value = 'All'
    else:
        cycles = sorted(df['payload.CurrentCycle'].dropna().unique().astype(int).tolist())
        cycles = [c for c in cycles if c > 0]  # ignore cycle 0
        steady_cycle_dropdown.options = ['All'] + cycles
        steady_cycle_dropdown.value = 'All'
        
    # call the plot after dropdown is updated
    plot_steady_state_patch()

steady_step_selector = widgets.SelectMultiple(options=['All'], value=['All'], description='Choose Step(s):', disabled=False,
                                              style={'description_width':'initial'}, rows=9, layout=widgets.Layout(width='300px'))

steady_cycle_dropdown = widgets.Dropdown(description='Choose Cycle:', style={'description_width':'initial'},
                                         layout=widgets.Layout(width='250px'))
update_steady_cycle_dropdown()
steady_cycle_dropdown.observe(plot_steady_state_patch, names='value')
steady_step_selector.observe(plot_steady_state_patch, names='value')
plot_steady_state_patch()

def print_header_footer_data():
    if df.empty:
        return

    header_row = df.iloc[0]
    guid = header_row.get("payload.GUID")
    username = header_row.get("payload.username")
    experimental_details = header_row.get("payload.experimentalDetails")
    valve_sequence = header_row.get("payload.valveSequence")
    start_time = header_row.get("payload.startTime")
    reactor_name = header_row.get("payload.reactorName")

    footer_row = df.iloc[-1]
    end_time = footer_row.get("payload.EndTime")

    print(f"Reactor: {reactor_name}\n")
    print(f"GUID: {guid}\n")
    print(f"User: {username}\n")
    print(f"Experimental Details: {experimental_details}\n")
    print(f"Start Time: {start_time}\n")
    print(f"End Time: {end_time}\n")

    if valve_sequence is not None:
        print("Valve Sequence:")
        for row in valve_sequence:
            formatted_row = " ".join(f"{x:>6}" for x in row)
            print(formatted_row)
    
# ╭──────────────────────────────────────────────────────────────────────────────────────────────────────────╮
# │                                    More Widgets and Updater Functions                                    │
# ╰──────────────────────────────────────────────────────────────────────────────────────────────────────────╯
pressure_output = widgets.Output()
max_pressure_output = widgets.Output()
steady_state_output = widgets.Output()
missing_pressure_output = widgets.Output()
header_footer_data_output = widgets.Output()
pressure_loading = widgets.Label(value='')

# steady-state parameters widgets
steady_frac_widget = widgets.BoundedFloatText(
    value=steady_frac, min=0.01, max=1.0, step=0.01,
    description='Steady Fraction:', style={'description_width':'initial'},
    layout=widgets.Layout(width='250px')
)

slope_tol_widget = widgets.BoundedFloatText(
    value=slope_tol_s, min=0.0, step=0.1,
    description='Slope Tolerance (mTorr/s):', style={'description_width':'initial'},
    layout=widgets.Layout(width='250px')
)

amp_tol_widget = widgets.BoundedFloatText(
    value=amp_tol, min=0.0, step=1.0,
    description='Peak-to-Peak Amp (mTorr):', style={'description_width':'initial'},
    layout=widgets.Layout(width='250px')
)

min_points_widget = widgets.BoundedIntText(
    value=min_points, min=2, step=1,
    description='Minimum Points:', style={'description_width':'initial'},
    layout=widgets.Layout(width='250px')
)

def update_pressure_plot(change):
    cycle_selection = change['new']
    pressure_loading.value = f'Loading cycle {cycle_selection}...' if cycle_selection!='All' else 'Loading all cycles...'
    with pressure_output:
        clear_output(wait=True)
        if not pressure_df.empty:
            plot_pressure(cycle_selection if cycle_selection!='All' else None)
    pressure_loading.value=''

def update_max_pressure(change=None):
    selected_steps = max_step_selector.value
    with max_pressure_output:
        clear_output(wait=True)
        if step_df.empty:
            print("No data")
            return
        df_show = step_df if 'All' in selected_steps else step_df[step_df['Step'].isin(selected_steps)]

        plt.figure(figsize=(6,6))
        for step in df_show['Step'].unique():
            subset = df_show[df_show['Step']==step]

            fill_color = step_dict[step]['color']
            outline_color = step_dict[step]['outline']
            marker = step_dict[step]['marker']
            
            plt.scatter(subset['Cycle'], subset['Max Pressure (mTorr)'], color=fill_color,
                        edgecolor=outline_color, linewidth=0.7, marker=marker, label=step)
        plt.title('Max Pressure by Step')
        plt.xlabel('Cycle')
        plt.ylabel('Max Pressure (mTorr)')
        plt.legend(title='Step', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.xlim(step_df["Cycle"].min(), step_df["Cycle"].max())
        plt.xticks(range(step_df["Cycle"].min(), step_df["Cycle"].max() + 1))
        
        ax = plt.gca()
        ax.yaxis.set_minor_locator(AutoMinorLocator())
        
        plt.show()
        display(df_show)

def update_steady_state(change=None):
    selected_steps = steady_step_selector.value
    with steady_state_output:
        clear_output(wait=True)
        if step_df.empty:
            print("No data")
            return
        df_show = step_df if 'All' in selected_steps else step_df[step_df['Step'].isin(selected_steps)]

        plt.figure(figsize=(6,6))
        for step in df_show['Step'].unique():
            subset = df_show[df_show['Step']==step]
            fill_color = step_dict[step]['color']
            outline_color = step_dict[step]['outline']
            marker = step_dict[step]['marker']
            plt.scatter(subset['Cycle'], subset['Steady Pressure (mTorr)'], color=fill_color,
                        edgecolor=outline_color, linewidth=0.7, marker=marker, label=step)

        plt.title('Steady-State Pressure by Step')
        plt.xlabel('Cycle')
        plt.ylabel('Steady Pressure (mTorr)')
        plt.legend(title='Step', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.xlim(step_df["Cycle"].min(), step_df["Cycle"].max())
        plt.xticks(range(step_df["Cycle"].min(), step_df["Cycle"].max() + 1))

        ax = plt.gca()
        ax.yaxis.set_minor_locator(AutoMinorLocator())
        
        plt.show()
        display(df_show)

cycle_dropdown.observe(update_pressure_plot, names='value')
max_step_selector.observe(update_max_pressure, names='value')
steady_step_selector.observe(update_steady_state, names='value')

def update_steady_params(change):
    global steady_frac, slope_tol_s, amp_tol, min_points
    steady_frac = steady_frac_widget.value
    slope_tol_s = slope_tol_widget.value
    amp_tol = amp_tol_widget.value
    min_points = min_points_widget.value

    # recompute steady pressure and slope
    compute_steady_pressures_and_slopes()
    plot_steady_state_patch() # refresh steady state patch plot
    update_steady_state()     # refresh steady state scatterplot

for w in [steady_frac_widget, slope_tol_widget, amp_tol_widget, min_points_widget]:
    w.observe(update_steady_params, names='value')

def update_missing_pressure():
    missing_pressure_output.clear_output(wait=True)
    with missing_pressure_output:
        if pressure_df.empty: print("No pressure data loaded yet.")
        else:
            nan_count = pressure_df["payload.Pressure"].isna().sum()
            print(f"The number of missing pressure packets from the original JSON file is {nan_count}.")

def update_header_footer_data():
    header_footer_data_output.clear_output(wait=True)
    with header_footer_data_output:
        print_header_footer_data()

def update_step_selectors():
    options = ['All'] + step_df['Step'].dropna().unique().tolist() if not step_df.empty else ['All']
    max_step_selector.options = options
    steady_step_selector.options = options
    
    if 'All' in max_step_selector.options:
        max_step_selector.value = ['All']
    if 'All' in steady_step_selector.options:
        steady_step_selector.value = ['All']

def update_all_tabs(change=None):
    update_cycle_dropdown()
    update_step_selectors()
    update_pressure_plot({'new': cycle_dropdown.value})
    update_max_pressure()
    update_steady_state()
    update_missing_pressure()
    update_header_footer_data()
    update_pressure_overlay()
    update_steady_cycle_dropdown()

# ╭──────────────────────────────────────────────────────────────────────────────────────────────────────────╮
# │                         Descriptions, Misc Info Tab, Descriptions, GUI Display, etc.                     │
# ╰──────────────────────────────────────────────────────────────────────────────────────────────────────────╯
misc_info_text = """
<b><u>Steady-State Tab Information</u></b><br>

The <i>Steady-State</i> tab uses four criteria to determine if a step has reached steady state:<br>

1. <b><u>Steady Fraction</u></b>:<br>
   - The fraction of a step's final points to use for the steady-state calculation.<br>
   - Example: 0.2 means only the last 20% of a step's points are analyzed.<br>

2. <b><u>Slope Tolerance</u></b>:<br> 
   - The maximum allowed slope of a least-squares line fitted to the selected points.<br>
   - A slope exceeding this value has not reached steady state (dP/dt≈0).<br>

3. <b><u>Peak-to-Peak Amplitude Tolerance</u></b>:<br>
   - The maximum difference between the highest and lowest points in the selected fraction.<br>
   - Large fluctuations prevent the step from being considered steady, even if the slope is small.<br>

4. <b><u>Minimum Number of Points in Step</u></b>:<br>
   - A step must contain at least the designated minimum number of points.<br>
<br>
Note: For <i>Max Plot</i> and <i>Steady-State Plot</i>, multiple steps can be selected with shift and/or
ctrl (or command) pressed and mouse clicks or arrow keys.
<br><br>
For the color-coded pressure plot under the Pressure Plot tab, selecting a cycle or using the Previous/Next Cycle buttons is somewhat
slow. This is because a new plot is generated each time the cycle changes, rather than simply updating the x-axis limits. Modifying the
plot_pressure function to update only the axis bounds in the future would improve computational efficiency and greatly reduce the
wait time for cycle updates.<br>
<br>
<b><u>Note</b></u>: Use <b>Upload File</b> only for smaller files (tested up to ~2.5MB).<br>
For larger files (e.g., 30MB+), use <b>File Path</b> instead since it's much faster and won't stall.
"""

misc_info_output = widgets.Output()
with misc_info_output:
    display(widgets.HTML(misc_info_text))

# descriptions
pressure_desc = widgets.HTML("<b>Pressure Plot:</b> Shows pressure vs. time for the selected cycle by step.")
max_pressure_desc = widgets.HTML("<b>Max Pressure:</b> Shows max pressure vs. cycle by step.")
steady_state_desc = widgets.HTML("<b>Steady-State Pressure:</b> Shows steady-state pressure vs. cycle by step.")
missing_pressure_desc = widgets.HTML("<b>Missing Pressure Test:</b> Counts missing pressure data from the loaded JSON file.")
header_footer_data_desc = widgets.HTML("<b>Experimental Info:</b>")

steady_widgets_box = widgets.HBox([steady_frac_widget, slope_tol_widget, amp_tol_widget, min_points_widget])

spacer = widgets.Label(value='', layout=widgets.Layout(height='20px'))

tab = widgets.Tab(children=[
    widgets.VBox([pressure_desc,
                  cycle_dropdown,
                  pressure_loading,
                  pressure_output,
                  cycle_selector_box,
                  spacer,
                  overlay_accordion]),
    widgets.VBox([max_pressure_desc,
                  max_step_selector,
                  max_pressure_output]),
    widgets.VBox([steady_state_desc,
                  steady_cycle_dropdown,
                  steady_state_patch_output,
                  steady_widgets_box,
                  steady_step_selector,
                  steady_state_output]),
    widgets.VBox([header_footer_data_desc,
                  header_footer_data_output]),
    widgets.VBox([missing_pressure_desc,
                  missing_pressure_output]),
    misc_info_output])

tab.set_title(0, 'Pressure Plot')
tab.set_title(1, 'Max Plot')
tab.set_title(2, 'Steady-State Plot')
tab.set_title(3, 'Experiment Data')
tab.set_title(4, 'Missing Data Test')
tab.set_title(5, 'Misc. Info')

display(tab)

In [ ]:
# df

In [ ]:
# pressure_df

In [ ]:
# step_df